In [5]:
import pdfplumber
import pandas as pd
from deep_translator import GoogleTranslator
import PyPDF2
import re
import operator
import numpy as np
    
pdf = pdfplumber.open("france.pdf")

def keywords():
    questions_to_keywords={
    "holiday":"Can your whole household afford to go for a week’s annual holiday, away from home?",
    "vegetarian":"Can your household afford a meal with meat, chicken, fish(or vegetarian equivalent)?",
    "expense": "Can your household afford an unexpected required expense(amount to be filled) and pay through its own resources?",
    "telephone":"Does your household have a telephone(fixed landline or mobile)?",
    "colour TV":"Does your household have a color TV?",
    "washing machine":"Does the household have a washing machine? ",
    "van":"Does your household have a car/van for private use? ",
    "dwelling":"Do you have any of the following problems with your dwelling/accommodation? ",
    "warm":"Can your household afford to keep its home adequately warm?  "
} 
    
    translated_keywords = []
    for key in questions_to_keywords.keys():
        translated = GoogleTranslator(source='en', target='french').translate(key)
        translated_keywords.append(translated)
    return translated_keywords

    
    
translated_keywords = keywords()
print(translated_keywords)

#testing regex - re.IGNORECASE works so it should work when matching keywords but doesnt for somre rason
def test_regex():
    matched = []
    matched.append(re.findall('hé',"héllo"))
    return matched

print(test_regex())

                               
#function to translate from detected language into english using deep translator library 
def translator(lines):
    translated_array =[]
    for i in lines:
        to_translate = i
        translated = GoogleTranslator(source='auto', target='en').translate(i)
        translated_array.append(translated)
    return translated_array


#cleans text removing next lines, whitespace, etc 
def clean(text):
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    
    return text



#DO NOT CHANGE- FULLY IDENTIFIES QUESTIONS ON A PAGE
#given a page of a docuent or list of pages, will find all questions on that page and translate into english 
def translate_document(pages):
    pdf1 = pdfplumber.open("france.pdf")
    translated_array = []
    #pages = list(pages)
    # writing page 161 will translate page 162
    pages= pages
    for number in pages:
        p1 = pdf1.pages[number]
        im = p1.to_image()
        text = p1.extract_text()
        text = clean(text)
        #text = re.split('[?]',text)  
        text = re.findall('(?<=[\?\.\!]\s)[^\?\n\.]+?\?',text)
        clean_sent  = []
        for sent in text:
            clean_sent.append(sent) 
        #translated_array.append(clean_sent)
        translated_array.append(translator(clean_sent))
    return translated_array



pages = []
for word in translated_keywords:
    word = word.lower()
    pdf = PyPDF2.PdfFileReader(r"france.pdf")
    for i in range(0,pdf.getNumPages()):
        PageObj = pdf.getPage(i)
        Text = PageObj.extractText()     
        if re.findall(word,Text,re.IGNORECASE):
            #pages.append(word)
            pages.append(i) 
            
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


translatedDoc=translate_document(pages)

['vacances', 'végétarien', 'frais', 'Téléphone', 'téléviseur couleur', 'Machine à laver', 'van', 'logement', 'chaud']
[['hé']]


In [6]:

DftranslatedDoc=pd.DataFrame(translatedDoc)
DftranslatedDoc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,The address sheet was created by mistake If RE...,(Answer in clear)……………………………… Go to VALIDF If ...,Unusual and uncertain until the end of data co...,None,None,None,None,None,None,None,None,None,None,None,None
1,"» if BS > 0 (Y14) According to you, is the acc...",A leisure or holiday residence?,Occasional accommodation for studies or work?,None,None,None,None,None,None,None,None,None,None,None,None
2,Otherwise: TYPMEN Occupation of the same accom...,no If TYPMEN = 1 (The occupation is unchanged)...,More weekends or holidays (cid:159) (A9) Appro...,Mostly during the week (cid:159) (A10) How man...,A few months in the year (including the case o...,More rarely (cid:159) (A12) About how many day...,None,None,None,None,None,None,None,None,None
3,"If MOVEMENT = 2, Since what date has PRENOM be...",LNAIS Born in France/abroad DEPNAIS Department...,Abroad If LNAIS = 1 (France): (A5) In which ov...,coding by trigram DK REFUSAL If LNAIS = 2 (for...,coding by trigram with the Country table NSP R...,If ANARRIV = nsp: At what age did PRENOM settl...,almost all year round?,do you prefer weekends or holidays?,(cid:159) (A9) Approximately how many days per...,rather during the week?,(cid:159) (A10) How many days per week?,a few months in the year (including the case o...,(cid:159) (A11) How many months in a year?,more rarely ?,(cid:159) (A12) About how many days in a year?
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Did you [PRENOM perceive any] still in ^AN?,"If SITIND = (B), (C), (D) or (SITIND = (A) and...","No NSP REFUSAL If EXRMI=1 and if CAISSALAR =1,...",None,None,None,None,None,None,None,None,None,None,None,None
157,Has this benefit been paid?,To a person outside the current household DK R...,None,None,None,None,None,None,None,None,None,None,None,None,None
158,No NSP REFUSAL If TRAVAUX=2 and if V1WC=missin...,No NSP REFUSAL If TRAVAUX=2 and if V1EAUCHAUD=...,No NSP REFUSAL If TRAVAUX=2 and if V1CHAUF=mis...,None,None,None,None,None,None,None,None,None,None,None,None
159,No DK REFUSAL How much do you pay PER YEAR for...,"If possible, deduct the amount corresponding t...",Also include the cost of bottled gas If the ho...,None,None,None,None,None,None,None,None,None,None,None,None
